# Weighted doubly robust learning: A extended simulation

This simulation tests WDRL with five (imbalanced!) treatments, more sample, and more features. Some possible treatments are excluded as well.

In [4]:
from itertools import chain, combinations
import itertools

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

np.random.seed(1804)

In [5]:
n = 100_000
p = 100
b = 5  # number of treatments

X = np.random.uniform(0, 1, (n, p))

sigma = 2
error = np.random.normal(0, 1, n)

# Define propensity function g*(k) for treatment assignment
g_star = [.27, .17, .14, .27, .09]
T = np.array([np.random.binomial(1, g_star[k], n) for k in range(b)]).T

# baseline main effect e*(X_i)
e_star = np.sin(np.pi * X[:, 0] * X[:, 1]) + 2 * (X[:, 2] - 0.5) ** 2 + X[:, 3] + 0.5 * X[:, 4]

# treatment effect τ*(X_i)
tau_star = (X[:, 0] + X[:, 1]) * 0.5
# NOTE ^: Due to uniform distribution and this set up, the treatment effect of each will be about 0.5, the mean of each X

Y = e_star + (np.sum(T - 0.5, axis=1)) * tau_star + sigma * error

# Assemble into dataframe
df = pd.DataFrame(X)
df = df.rename(columns=lambda x: f"x{x}")

treatment_df = pd.DataFrame(T, columns=['TA', 'TB', 'TC', 'TD', 'TE'])
df = pd.concat([df, treatment_df], axis=1)

df['y'] = Y

# Create a joined treatment column using letters
def combine_treatments(row):
    treatments = []
    if row['TA'] == 1:
        treatments.append('A')
    if row['TB'] == 1:
        treatments.append('B')
    if row['TC'] == 1:
        treatments.append('C')
    if row['TD'] == 1:
        treatments.append('D')
    if row['TE'] == 1:
        treatments.append('E')
    return ''.join(treatments) if treatments else '0'

# Apply the function to each row and create a new column 't'
df['t'] = df.apply(combine_treatments, axis=1)



In [6]:
# Limit to specific treatments
df = df.query('t == "0" | t == "ABD" | t == "ABCDE" | t == "AD" | t == "ACD" | t == "ABCD" | t == "ABDE" | t == "A" | t == "ADE" | t == "BCD" | t == "BD" | t == "ACDE" | t == "D" | t == "BCDE" | t == "BDE" | t == "DE"')

len(df)

72389

In [7]:
# Set number of samples for each treatment
exact_sample_sizes = {'0': 36152,
                      'ABD': 2759, 
                      'ABCDE': 2745,
                      'AD': 2049,
                      'ACD': 1891,
                      'ABCD': 1643,
                      'ABDE': 1204,
                      'A': 629,
                      'ADE': 472,
                      'BCD': 350,
                      'BD': 232,
                      'ACDE': 229,
                      'D': 215,
                      'BCDE': 26,
                      'BDE': 25,
                      'DE': 3}

assert sum(exact_sample_sizes.values()) <= len(df), "Total sample size exceeds the number of rows in the dataframe."

# Sample data from each group
sampled_dfs = []
for treatment, n_samples in exact_sample_sizes.items():
    group = df[df['t'] == treatment]
    # Ensure we don't sample more than available in the group
    sampled_dfs.append(group.sample(n=min(len(group), n_samples), replace=False, random_state=42))

# Combine sampled dataframes
balanced_df = pd.concat(sampled_dfs, ignore_index=True)

balanced_df['t'].value_counts()


t
0        34824
AD        2049
ABD        950
ACD        783
A          629
ADE        472
BCD        350
BD         232
D          215
ABCD       166
ABDE       100
ACDE        84
BCDE        26
BDE         25
ABCDE       24
DE           3
Name: count, dtype: int64

In [8]:
# # Split into train/test
# df_train = df.iloc[:7000]
# df_test = df.iloc[7000:]

df_train = balanced_df

In [9]:
# Create treatment weights
treatment_prob = df_train['t'].value_counts(normalize=True).to_dict()

In [10]:
TREATMENTS = ['A', 'B', 'C', 'D', 'E']

all_combinations = list(chain.from_iterable(combinations(TREATMENTS, r) for r in range(len(TREATMENTS) + 1)))
all_combinations = [''.join(i) for i in all_combinations]

C = ['0' if i == '' else i for i in all_combinations]

In [11]:


# Function to calculate propensity scores
def calculate_propensity_scores(X, T):
    model_t = make_pipeline(StandardScaler(), LogisticRegression(penalty=None))
    model_t.fit(X, T)
    preds = model_t.predict_proba(X)[:, 1]
    return np.clip(preds, 0.01, 0.99)

# Function to fit outcome models and compute doubly robust scores
def calculate_doubly_robust_scores(X, y, T, ps):
    m0 = LinearRegression().fit(X[T == 0], y[T == 0])
    m1 = LinearRegression().fit(X[T == 1], y[T == 1])
    
    m0_hat = m0.predict(X)
    m1_hat = m1.predict(X)

    y_dr_1 = m1_hat + (T * (y - m1_hat)) / ps
    y_dr_0 = m0_hat + ((1 - T) * (y - m0_hat)) / (1 - ps)
    
    return y_dr_1 - y_dr_0

# Function to compute weighted doubly robust scores
def compute_weighted_dr_scores(dr_scores, treatment_prob):
    valid_weights = {col: treatment_prob[col] for col in dr_scores.columns if col in treatment_prob}
    weighted_sum = sum(dr_scores[col] * w for col, w in valid_weights.items())
    return weighted_sum / sum(valid_weights.values())

# Main loop to calculate ATE for each treatment
def calculate_ate(df_train, C, treatment_prob, treatments):
    treatment_cols = [f'T{t}' for t in treatments]
    X = df_train.drop(columns=['y', 't'] + treatment_cols)
    results = {}
    
    for treatment in treatments:
        print(f"\nProcessing treatment {treatment}...")
        
        # Split combinations into those with and without the current treatment
        W_k = [comb for comb in C if treatment in comb]
        S__k = [comb for comb in C if treatment not in comb]

        # Limit to existing pairs
        W_k = [w for w, s in zip(W_k, S__k) if w in treatment_prob.keys() and s in treatment_prob.keys()]
        S__k = [w.replace(treatment, '') for w in W_k]
        S__k = ['0' if s == '' else s for s in S__k]

        print(W_k)
        print(S__k)

        y_hat_dr_list = []
        
        for w_k, s__k in zip(W_k, S__k):

            print(f'{w_k} v. {s__k}') 

            # Create treatment/control datasets
            df_subset_w_k = df_train.query('t == @w_k').copy()
            df_subset_s__k = df_train.query('t == @s__k').copy()

            if len(df_subset_w_k) < 50 or len(df_subset_s__k) < 50:
                print(f'Insufficient sample for {w_k} v. {s__k}')
                continue
            
            df_subset = pd.concat([df_subset_w_k, df_subset_s__k])
            df_subset['t_binary'] = np.where(df_subset['t'] == w_k, 1, 0)
            
            X_treated = df_subset.drop(columns=['y', 't', 't_binary'] + treatment_cols)
            y_treated = df_subset['y']
            T_treated = df_subset['t_binary']
            
            # Calculate propensity scores and doubly robust scores
            ps = calculate_propensity_scores(X_treated, T_treated)
            dr_scores = calculate_doubly_robust_scores(X_treated, y_treated, T_treated, ps)
            
            # Fit DR model on treatment group and apply it to the full dataset
            dr_model = LinearRegression().fit(X_treated, dr_scores)
            # dr_model = make_pipeline(StandardScaler(), Lasso(alpha=.01, tol=1e-3, max_iter=5000, random_state=42)).fit(X_treated, dr_scores)
            y_hat_dr = dr_model.predict(X)
            y_hat_dr_list.append(y_hat_dr)

        # Aggregate results
        dr_df = pd.DataFrame({w_k: scores for w_k, scores in zip(W_k, y_hat_dr_list)})

        if len(dr_df) > 0:
            # Fit regression model to estimate ATE
            aggregated_dr_scores = compute_weighted_dr_scores(dr_df, treatment_prob)
            ate_model = LinearRegression().fit(X, aggregated_dr_scores)
            globals()['ate_model'] = ate_model
            globals()['ate_preds'] = ate_model.predict(X)
            globals()['ate_X'] = X
            globals()['ate_y'] = aggregated_dr_scores
            ate = np.mean(ate_model.predict(X))
        else:
            print(f'No sample for either {w_k} or {s__k}')
            ate = np.NAN
        
        results[treatment] = ate
        print(f"ATE for treatment {treatment}: {ate:.4f}")
    
    return results

# Example usage (replace with actual data and variables)
# df_train = ...
treatments = ['A', 'B', 'C', 'D', 'E']  # List of all treatment labels
# C = ...  # All possible combinations of treatments
# treatment_prob = ...  # Dictionary with treatment probabilities

ate_results = calculate_ate(df_train, C, treatment_prob, treatments)
print("Final ATE results:", ate_results)



Processing treatment A...
['A', 'AD', 'ABD', 'ADE', 'ABCD', 'ABDE', 'ABCDE']
['0', 'D', 'BD', 'DE', 'BCD', 'BDE', 'BCDE']
A v. 0
AD v. D
ABD v. BD
ADE v. DE
Insufficient sample for ADE v. DE
ABCD v. BCD
ABDE v. BDE
Insufficient sample for ABDE v. BDE
ABCDE v. BCDE
Insufficient sample for ABCDE v. BCDE
ATE for treatment A: 0.4357

Processing treatment B...
['BD', 'ABD', 'BDE', 'ABCD', 'ABDE', 'ABCDE']
['D', 'AD', 'DE', 'ACD', 'ADE', 'ACDE']
BD v. D
ABD v. AD
BDE v. DE
Insufficient sample for BDE v. DE
ABCD v. ACD
ABDE v. ADE
ABCDE v. ACDE
Insufficient sample for ABCDE v. ACDE
ATE for treatment B: 1.3888

Processing treatment C...
['ACD', 'BCD', 'ABCD', 'ACDE', 'BCDE', 'ABCDE']
['AD', 'BD', 'ABD', 'ADE', 'BDE', 'ABDE']
ACD v. AD
BCD v. BD
ABCD v. ABD
ACDE v. ADE
BCDE v. BDE
Insufficient sample for BCDE v. BDE
ABCDE v. ABDE
Insufficient sample for ABCDE v. ABDE
ATE for treatment C: 0.5179

Processing treatment D...
['D', 'AD']
['0', 'A']
D v. 0
AD v. A
ATE for treatment D: 0.5124

Proces

In [12]:
print("Final ATE results:", ate_results)

Final ATE results: {'A': np.float64(0.43566336149712215), 'B': np.float64(1.3887821582630853), 'C': np.float64(0.5178861664683317), 'D': np.float64(0.5124039550140255), 'E': np.float64(8.006757989955704)}
